<a href="https://colab.research.google.com/github/adeemadil/Colab-Notebooks/blob/main/Machine_Learning_HW1_Multinomial_Naive_Bayes_Classifier_(_Ham_or_Spam).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Importing Libraries***

In [ ]:
import os
import csv
import math
import random
import operator
import pdb

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

***Loading Dataset***

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

!ls /gdrive/MyDrive/ML_HW1

Mounted at /gdrive
vocabulary.txt	x_test.csv  x_train.csv  y_test.csv  y_train.csv


***Reading Dataset***

In [ ]:
root = '/gdrive/My Drive/ML_HW1'
csv_pathYTrain = os.path.join(root, 'y_train.csv')
csv_pathXTrain = os.path.join(root, 'x_train.csv')
csv_pathYTest = os.path.join(root, 'y_test.csv')
csv_pathXTest = os.path.join(root, 'x_test.csv')


In [ ]:
df_yTrain = pd.read_csv( csv_pathYTrain, header=None)

In [ ]:
df_xTrain = pd.read_csv( csv_pathXTrain, header=None)

In [ ]:
df_xTest = pd.read_csv( csv_pathXTest, header=None)


In [ ]:
df_yTest = pd.read_csv( csv_pathYTest, header=None)

In [ ]:
df_xTrain

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,43980,43981,43982,43983,43984,43985,43986,43987,43988,43989,43990,43991,43992,43993,43994,43995,43996,43997,43998,43999,44000,44001,44002,44003,44004,44005,44006,44007,44008,44009,44010,44011,44012,44013,44014,44015,44016,44017,44018,44019
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4081,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4082,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4083,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_yTrain

,0
0,1
1,1
2,1
3,0
4,0
...,...
4080,0
4081,1
4082,0
4083,1


In [ ]:
df_xTest

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,43980,43981,43982,43983,43984,43985,43986,43987,43988,43989,43990,43991,43992,43993,43994,43995,43996,43997,43998,43999,44000,44001,44002,44003,44004,44005,44006,44007,44008,44009,44010,44011,44012,44013,44014,44015,44016,44017,44018,44019
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1082,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1083,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1084,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_yTest

,0
0,1
1,1
2,0
3,1
4,0
...,...
1081,0
1082,1
1083,0
1084,1


# *Analysing the Data*

In [ ]:
spam = 0
normal = 0
index_y_train = df_yTrain.index
rows_y_train = len(index_y_train)
for i in range(rows_y_train):
  if df_yTrain[0].values[i] == 1:
    spam = spam + 1
  else:
    normal = normal + 1

probability_percent = round((100*(spam)/(spam+normal)),2)
print('The no. of spam emails= ' + str(spam))
print('The no. of non-spam emails= ' + str(normal))
print('The % of spam emails are ', probability_percent)

The no. of spam emails= 2911
The no. of non-spam emails= 1174
The % of spam emails are  71.26


# *MLE Estimators*

In [ ]:
def indexEmails(df_y):
  emailDict = {
      'spam' : [],
      'normal' : []
  }

  emailDict['spam'] = (df_y[df_y[0] == 1].index.to_numpy())
  emailDict['normal'] = (df_y[df_y[0] == 0].index.to_numpy())
  return emailDict

dictEmail = indexEmails(df_yTrain)
print(dictEmail['spam'])
print(dictEmail['normal'])

[   0    1    2 ... 4081 4083 4084]
[   3    4    7 ... 4078 4080 4082]


In [ ]:
df_xTrain.take(dictEmail['spam'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,43980,43981,43982,43983,43984,43985,43986,43987,43988,43989,43990,43991,43992,43993,43994,43995,43996,43997,43998,43999,44000,44001,44002,44003,44004,44005,44006,44007,44008,44009,44010,44011,44012,44013,44014,44015,44016,44017,44018,44019
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4081,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4083,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_xTrainSpam = df_xTrain.take(dictEmail['spam'])
df_xTrainNormal = df_xTrain.take(dictEmail['normal'])

In [ ]:
xTrainArraySpam = df_xTrainSpam.values
CountSpamWords = np.sum(xTrainArraySpam, axis=0)
print(CountSpamWords)

[1419 1508    3 ...    0    0    0]


In [ ]:
xTrainArrayNormal = df_xTrainNormal.values
CountNormalWords = np.sum(xTrainArrayNormal, axis=0)
print(CountNormalWords)

[471 187  16 ...   1   1   1]


Updating the Classifiers to compute and MAP Estimate using Drichlet prior

In [ ]:
##Remove these two lines of codes to not take MAP Estimate
CountNormalWords += 1
CountSpamWords += 1

In [ ]:
def findProbabilities(normalWords = [], spamWords = []):
  total_words_in_spam = np.sum(spamWords)
  total_words_in_normal = np.sum(normalWords)
  normalArr = normalWords.copy()
  spamArr = spamWords.copy()

  normalArr = normalArr / total_words_in_normal
  spamArr = spamArr / total_words_in_spam

  return spamArr, normalArr 

particularWordSpamProbability, particularWordNormalProbability = findProbabilities(CountNormalWords, CountSpamWords)
print('P(Xj | Y = spam)', particularWordSpamProbability)
print('P(Xj | Y = positive)', particularWordNormalProbability)

priorSpamProb = probability_percent/100
priorNormalProb = 1-priorSpamProb
print('P(Y = spam) = ', priorSpamProb)
print('P(Y = normal) = ', priorNormalProb)

P(Xj | Y = spam) [4.39034495e-03 4.66551446e-03 1.23671689e-05 ... 3.09179222e-06
 3.09179222e-06 3.09179222e-06]
P(Xj | Y = positive) [2.60488526e-03 1.03753905e-03 9.38200201e-05 ... 1.10376494e-05
 1.10376494e-05 1.10376494e-05]
P(Y = spam) =  0.7126
P(Y = normal) =  0.2874


In [ ]:
def calcProbability( wordSpamProb = [], wordNormalProb = []):
  totalProbSpam = 0
  totalProbNormal = 0

  totalProbSpam += np.log(wordSpamProb)
  totalProbNormal += np.log(wordNormalProb)

  return totalProbSpam, totalProbNormal

In [ ]:
probSpamGivenX, probNormalGivenX = calcProbability( particularWordSpamProbability, particularWordNormalProbability)
print( 'P( Y=spam | Xj)', np.nan_to_num(probSpamGivenX))
print( 'P( Y=normal | Xj)', np.nan_to_num(probNormalGivenX))

P( Y=spam | Xj) [ -5.42834748  -5.36755717 -11.30046527 ... -12.68675963 -12.68675963
 -12.68675963]
P( Y=normal | Xj) [ -5.95036665  -6.87090367  -9.27413229 ... -11.41419845 -11.41419845
 -11.41419845]


 ***Classifying the Data***

In [ ]:
xTestArr = df_xTest.to_numpy()

In [ ]:
xTestArr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
yTestArr = df_yTest.to_numpy()

In [ ]:
yTestArr = yTestArr.ravel()

In [ ]:
probSpamList = []
x = np.zeros(len(probSpamGivenX))
for email in xTestArr:
  np.multiply(probSpamGivenX, email, out=x)
  probSpamList.append( np.nan_to_num(x))
print(len(probSpamList))

1086


In [ ]:
spamProbSum = np.nan_to_num(np.sum(probSpamList, 1))
totalProbSpam = math.log(priorSpamProb) + spamProbSum
print(totalProbSpam)

[ -968.47395316  -574.75870047  -675.8935266  ... -1127.24574217
  -407.25908985  -662.61480663]


In [ ]:
probNormalList = []
y = np.zeros(len(probNormalGivenX))
#print(len(y))
for email in xTestArr:
  np.multiply(probNormalGivenX, email, out=y)
  probNormalList.append( np.nan_to_num(y))
print(probNormalList)

In [ ]:
normalProbSum = np.nan_to_num(np.sum(probNormalList, 1))
totalProbNormal = math.log(priorNormalProb) + normalProbSum
print(totalProbNormal)

[-994.6423508  -756.75198951 -541.41496918 ... -980.02032803 -579.24428993
 -917.19870106]


In [ ]:
totalProbSpam = np.exp(totalProbSpam)
totalProbNormal = np.exp(totalProbNormal)

In [ ]:
predictions = (totalProbSpam > totalProbNormal).astype(int)
#0 - normal , 1- spam

## **Accuracy and Confusion Matrix**

Confusion Matrix

In [ ]:
def printConfusionMatrix( predicted, actual):
  data = {'y_Actual':    actual,
        'y_Predicted': predicted
        }

  df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])

  confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
  return confusion_matrix
print (printConfusionMatrix( predictions, yTestArr))

Predicted    0    1
Actual             
0          315   10
1          222  539


Accuracy from Confusion Matrix data

In [ ]:
def getAccuracy( TN, FP, FN, TP):
  accuracy = (TN+TP)/(TN+FP+FN+TP)
  return accuracy

accuracy = getAccuracy(315, 10, 222, 539)
print( 'The accuracy of test computed, via tha data from Confusion matrix= %.2f'%accuracy)

The accuracy of test computed, via tha data from Confusion matrix= 0.79


# *Bag-of-Words Representation and Bernoulli Naive Bayes Model*

MLE ESTIMATORS

In [ ]:
xArrNormal = xTrainArrayNormal > 0
xArrNormal = xArrNormal.astype(int)
print(xArrNormal)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
xArrSpam = xTrainArraySpam > 0
xArrSpam = xArrSpam.astype(int)
print(xArrSpam)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
 countSpamWord = np.sum(xArrSpam, axis=0)
 countNormalWord = np.sum(xArrNormal, axis=0)
 print(countSpamWord)
 print(countNormalWord)

[498 606   2 ...   0   0   0]
[104  90   4 ...   1   1   1]


In [ ]:
n_Spam = len(df_xTrainSpam) #2911
n_Normal = len(df_xTrainNormal) #1174
n_total_email = n_Spam + n_Normal #4085

In [ ]:
def findBNProbab(normalWords = countNormalWord, spamWords = countSpamWord, 
                 spamEmails = n_Spam, normalEmails = n_Normal):
  normalArr = normalWords.copy()
  spamArr = spamWords.copy()
  totalEmail = normalEmails + spamEmails

  normalArr = normalArr / n_Normal
  spamArr = spamArr / n_Spam
  normalEmailsProb =  normalEmails/ totalEmail

  return spamArr, normalArr, normalEmailsProb

In [ ]:
spamArrProb, normalArrProb, normalEmailsProb = findBNProbab()
print("P( Xj | Y=spam): ", spamArrProb)
print("P( Xj | Y=normal): ", normalArrProb)
print("P( Y=normal): ", normalEmailsProb)

P( Xj | Y=spam):  [0.17107523 0.20817588 0.00068705 ... 0.         0.         0.        ]
P( Xj | Y=normal):  [0.08858603 0.07666099 0.00340716 ... 0.00085179 0.00085179 0.00085179]
P( Y=normal):  0.28739290085679314


Classifying Data

In [ ]:
#Bernoulli Matrix
testArrayX = xTestArr > 0
testArrayX = testArrayX.astype(int)
print(testArrayX)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
SpamProbabilitites = testArrayX * spamArrProb
print( SpamProbabilitites)

normalProbabilitites = testArrayX * normalArrProb
print( normalProbabilitites)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
complementTestArray = 1 - testArrayX
print(complementTestArray)

[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [ ]:
complementSpamProbabilitites = complementTestArray * (1-spamArrProb)
print(complementSpamProbabilitites)

complementNormalProbabilitites = complementTestArray * (1-normalArrProb)
print(complementNormalProbabilitites)

[[0.82892477 0.79182412 0.99931295 ... 1.         1.         1.        ]
 [0.82892477 0.79182412 0.99931295 ... 1.         1.         1.        ]
 [0.82892477 0.79182412 0.99931295 ... 1.         1.         1.        ]
 ...
 [0.82892477 0.79182412 0.99931295 ... 1.         1.         1.        ]
 [0.82892477 0.79182412 0.99931295 ... 1.         1.         1.        ]
 [0.82892477 0.79182412 0.99931295 ... 1.         1.         1.        ]]
[[0.91141397 0.92333901 0.99659284 ... 0.99914821 0.99914821 0.99914821]
 [0.91141397 0.92333901 0.99659284 ... 0.99914821 0.99914821 0.99914821]
 [0.91141397 0.92333901 0.99659284 ... 0.99914821 0.99914821 0.99914821]
 ...
 [0.91141397 0.92333901 0.99659284 ... 0.99914821 0.99914821 0.99914821]
 [0.91141397 0.92333901 0.99659284 ... 0.99914821 0.99914821 0.99914821]
 [0.91141397 0.92333901 0.99659284 ... 0.99914821 0.99914821 0.99914821]]


In [ ]:
logToBEProbSpam = (SpamProbabilitites) + (complementSpamProbabilitites)
afterLogprobSpam = np.nan_to_num(np.log( logToBEProbSpam))
print(afterLogprobSpam)

logToBEProbNormal = (normalProbabilitites) + (complementNormalProbabilitites)
afterLogprobNormal = np.nan_to_num(np.log( logToBEProbNormal))
print(afterLogprobNormal)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


[[-0.18762588 -0.23341599 -0.00068729 ...  0.          0.
   0.        ]
 [-0.18762588 -0.23341599 -0.00068729 ...  0.          0.
   0.        ]
 [-0.18762588 -0.23341599 -0.00068729 ...  0.          0.
   0.        ]
 ...
 [-0.18762588 -0.23341599 -0.00068729 ...  0.          0.
   0.        ]
 [-0.18762588 -0.23341599 -0.00068729 ...  0.          0.
   0.        ]
 [-0.18762588 -0.23341599 -0.00068729 ...  0.          0.
   0.        ]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


[[-0.09275807 -0.07975882 -0.00341297 ... -0.00085215 -0.00085215
  -0.00085215]
 [-0.09275807 -0.07975882 -0.00341297 ... -0.00085215 -0.00085215
  -0.00085215]
 [-0.09275807 -0.07975882 -0.00341297 ... -0.00085215 -0.00085215
  -0.00085215]
 ...
 [-0.09275807 -0.07975882 -0.00341297 ... -0.00085215 -0.00085215
  -0.00085215]
 [-0.09275807 -0.07975882 -0.00341297 ... -0.00085215 -0.00085215
  -0.00085215]
 [-0.09275807 -0.07975882 -0.00341297 ... -0.00085215 -0.00085215
  -0.00085215]]


In [ ]:
data = np.zeros(( len(afterLogprobSpam), len(afterLogprobSpam[0])))
data += (1-normalEmailsProb)
predictedProbsSpam = afterLogprobSpam + np.log(data)

data = np.zeros(( len(afterLogprobNormal), len(afterLogprobNormal[0])))
data += normalEmailsProb
predictedProbsNormal = afterLogprobNormal + np.log(data)

In [ ]:
print(predictedProbsSpam)
print( predictedProbsNormal)

[[-0.52645094 -0.57224105 -0.33951235 ... -0.33882506 -0.33882506
  -0.33882506]
 [-0.52645094 -0.57224105 -0.33951235 ... -0.33882506 -0.33882506
  -0.33882506]
 [-0.52645094 -0.57224105 -0.33951235 ... -0.33882506 -0.33882506
  -0.33882506]
 ...
 [-0.52645094 -0.57224105 -0.33951235 ... -0.33882506 -0.33882506
  -0.33882506]
 [-0.52645094 -0.57224105 -0.33951235 ... -0.33882506 -0.33882506
  -0.33882506]
 [-0.52645094 -0.57224105 -0.33951235 ... -0.33882506 -0.33882506
  -0.33882506]]
[[-1.33966308 -1.32666383 -1.25031798 ... -1.24775716 -1.24775716
  -1.24775716]
 [-1.33966308 -1.32666383 -1.25031798 ... -1.24775716 -1.24775716
  -1.24775716]
 [-1.33966308 -1.32666383 -1.25031798 ... -1.24775716 -1.24775716
  -1.24775716]
 ...
 [-1.33966308 -1.32666383 -1.25031798 ... -1.24775716 -1.24775716
  -1.24775716]
 [-1.33966308 -1.32666383 -1.25031798 ... -1.24775716 -1.24775716
  -1.24775716]
 [-1.33966308 -1.32666383 -1.25031798 ... -1.24775716 -1.24775716
  -1.24775716]]


In [ ]:
totalPredictedSpamProb = np.nan_to_num(  predictedProbsSpam.sum(axis=1))
print(totalPredictedSpamProb)

[-1.79769313e+308 -1.52170232e+004 -1.79769313e+308 ... -1.79769313e+308
 -1.79769313e+308 -1.51861085e+004]


/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [ ]:
totalPredictedNormalProb = np.nan_to_num(  predictedProbsNormal.sum(axis=1))
print(totalPredictedNormalProb)

[-1.79769313e+308 -1.79769313e+308 -5.51226126e+004 ... -5.53054518e+004
 -1.79769313e+308 -1.79769313e+308]


/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


Make Predictions, Confusion Matrix and Accuracy


In [ ]:
predictionsBN = (totalPredictedSpamProb > totalPredictedNormalProb).astype(int)

In [ ]:
print (printConfusionMatrix( predictionsBN, yTestArr))

Predicted    0    1
Actual             
0          300   25
1          148  613


In [ ]:
accuracyBN = getAccuracy(300, 25, 148, 613)
print( 'The accuracy of test computed, via tha data from Confusion matrix= %.2f'%accuracyBN)

The accuracy of test computed, via tha data from Confusion matrix= 0.84
